In [1]:
# Functionality for computing features
from pyspark.ml import feature
# Functionality for regression
from pyspark.ml import regression
# Funcionality for classification
from pyspark.ml import classification
# Object for creating sequences of transformations
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
import pandas as pd

# Section 1: Slice Data, United States
Dropped a few unnecessary columns,
Filter the dataset by United States

In [2]:
#data_01 = pd.read_csv('Data_clean_01.csv', sep = ',')
#data_01 is the step after 1st step of the data cleaning
#instead of reading into Panda, we directly passed it into pyspark

In [3]:
df01 = spark.read.csv('Data_clean_1.csv',sep = ',', header=True, inferSchema=True)
#read the dataset into spark

In [4]:
df01.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- channelGrouping: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- fullVisitorId: decimal(20,0) (nullable = true)
 |-- socialEngagementType: string (nullable = true)
 |-- visitId: integer (nullable = true)
 |-- visitNumber: integer (nullable = true)
 |-- device.browser: string (nullable = true)
 |-- device.deviceCategory: string (nullable = true)
 |-- device.isMobile: boolean (nullable = true)
 |-- device.language: string (nullable = true)
 |-- device.mobileDeviceBranding: string (nullable = true)
 |-- device.mobileDeviceModel: string (nullable = true)
 |-- device.operatingSystem: string (nullable = true)
 |-- device.screenResolution: string (nullable = true)
 |-- geoNetwork.city: string (nullable = true)
 |-- geoNetwork.continent: string (nullable = true)
 |-- geoNetwork.country: string (nullable = true)
 |-- geoNetwork.latitude: string (nullable = true)
 |-- geoNetwork.longitude: string (nullable = true)
 |-- totals.bounc

In [5]:
#df 02 is the datafram that after dropping columns that we deems unhelpful here
#there other columns whose information isn't available, wiped by Google to protect privacy
#we couldn't process the geo information at this point, that's why we dropped them, city, country & continent
#latitude & longtitude aren't available to us
df02 = df01.drop("_c0", "device.language","device.mobileDeviceBranding","device.mobileDeviceModel",
                 "device.screenResolution", "geoNetwork.city","geoNetwork.continent", #"geoNetwork.country",
                 "geoNetwork.latitude", "geoNetwork.longitude")

In [6]:
print(df02.columns)

['channelGrouping', 'date', 'fullVisitorId', 'socialEngagementType', 'visitId', 'visitNumber', 'device.browser', 'device.deviceCategory', 'device.isMobile', 'device.operatingSystem', 'geoNetwork.country', 'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews', 'totals.sessionQualityDim', 'totals.timeOnSite', 'totals.totalTransactionRevenue', 'totals.transactionRevenue', 'totals.transactions', 'totals.visits', 'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect', 'trafficSource.medium', 'trafficSource.source']


In [7]:
df02.count()

1708337

In [8]:
df02.select("`geoNetwork.country`").show(2)
#here, i need a special backticks to wrap up the column name so it'll work 

+------------------+
|geoNetwork.country|
+------------------+
|           Germany|
|     United States|
+------------------+
only showing top 2 rows



In [9]:
#df02.select("geoNetwork.country").
#df02.filter("geoNetwork.country='united states'").show()
#df2.select([c for c in df02.geoNetwork.country if c in ['united states']]).show()
#A link for filtering out rows
#https://stackoverflow.com/questions/49301373/pyspark-filter-dataframe-based-on-multiple-conditions?rq=1
import pyspark.sql.functions as fn
df03 = df02.filter((fn.col('`geoNetwork.country`') == "United States"))
# df03 is the dataframe that has only the data for United States 

In [10]:
df03.count()

717217

# Section 2: Feature Enigeering
This is the section to explore each feature
Put them into categorical groups for the pipe line. 
df03 is the input dataset for this section

In [11]:
#renames all the columns, 
#have been running into bugs becaues of how columns were named 
df03 = df03.toDF('channelGrouping','date','fullVisitorId','socialEngagementType','visitId',
                 'visitNumber','browser','deviceCategory','isMobile','operatingSystem','country',
                 'totalsbounces','totalshits','totalsnewVisits', 'totalspageviews',
                 'totalssessionQualityDim', 'totalstimeOnSite', 'totalTransactionRevenue',
                 'transactionRevenue', 'transactions', 'totalsvisits', 'adwordsClickInfoisVideoAd',
                 'isTrueDirect','trafficSourcemedium', 'trafficSourcesource')

In [12]:
#df03.printSchema()
df04 = df03 
#df 04 backs up df03

## Feature: ChannelGrouping 
7 classes + Null&others together

In [13]:
#count if there are any NAs in the column 
# from pyspark.sql.functions import isnan This is for checking NaN, if necessary 
# df.filter( (df["ID"] == "") | (df["ID"].isNull()) | ( df["ID"].isnan()) ).count()
# this is for "", or Null, or NaN
df04.filter(df04["channelGrouping"].isNull()).count()
#df03.select("channelGrouping").na.fill("Other").cube("channelGrouping").count().show() 
#An earlier code tying to corrent null 

0

In [14]:
#here, Nullable is true, that's why we have 717217 null exactly mathces how many rows there are 
df04.cube("channelGrouping").count().sort("count").show()

+---------------+------+
|channelGrouping| count|
+---------------+------+
|        (Other)|   105|
|     Affiliates|  6985|
|        Display| 38579|
|    Paid Search| 39291|
|         Social| 45750|
|         Direct|141625|
|       Referral|149149|
| Organic Search|295733|
|           null|717217|
+---------------+------+



## Feature: Date
Drop the date column, because we aren't processing any time series data

In [15]:
df04 = df04.drop("date")

In [16]:
#df04.printSchema()

## Feature: fullVistorId,visitId are IDs, not need for processing
## Feature: visitNumber is similar 

## Feature: socialEngagmentType
Everyone is not socially engaged, the feature is useless.
We drop it. 

In [17]:
df04.cube("socialEngagementType").count().sort("count").show()

+--------------------+------+
|socialEngagementType| count|
+--------------------+------+
|Not Socially Engaged|717217|
|                null|717217|
+--------------------+------+



In [18]:
df04 = df04.drop("socialEngagementType")

## Feature: browser
### df04 before this Step
Here, we preserved 3 types. 
<br /> Chrome, Safari & Others
<br /> After this step, it's df05


In [19]:
#check wethere there are any nulls 
df04.filter(df04["browser"].isNull()).count()

0

In [20]:
from pyspark.sql.functions import desc
df04.cube("browser").count().sort(desc("count")).show()

+--------------------+------+
|             browser| count|
+--------------------+------+
|                null|717217|
|              Chrome|529385|
|              Safari|127216|
|             Firefox| 15247|
|   Internet Explorer| 11974|
|                Edge|  8420|
|     Safari (in-app)|  7369|
|    Samsung Internet|  7169|
|     Android Webview|  6242|
|               Opera|  2165|
|         Amazon Silk|  1070|
|Mozilla Compatibl...|   182|
|           YaBrowser|   169|
|          UC Browser|   117|
|     Android Browser|    86|
|          Opera Mini|    84|
|    Nintendo Browser|    46|
|              Puffin|    44|
|             Coc Coc|    41|
|             Maxthon|    35|
+--------------------+------+
only showing top 20 rows



In [21]:
#to make nullable = True 
df05 = df04.withColumn('Nbrowsers', fn.when(fn.col('browser')=='Chrome', 'Chrome').\
                                       when(fn.col('browser')=='Safari', 'Safari').otherwise('others')).\
                                       drop('browser')
#fn.when(fn.col('browser') =='Safari', 'Safari').\
#                            fn.when(fn.col('browser')=='Firefox', 'Firefox').\
#                            fn.when(fn.col('browser')=='Internet Explorer', 'IE').\

In [22]:
#df04_brs.printSchema()

In [23]:
df05.cube("Nbrowsers").count().sort(desc("count")).show()

+---------+------+
|Nbrowsers| count|
+---------+------+
|     null|717217|
|   Chrome|529385|
|   Safari|127216|
|   others| 60616|
+---------+------+



## Feature: deviceCategory

In [24]:
df05.cube("deviceCategory").count().sort(desc("count")).show()
#717217 is the total number of rows in the United States 

+--------------+------+
|deviceCategory| count|
+--------------+------+
|          null|717217|
|       desktop|475113|
|        mobile|210890|
|        tablet| 31214|
+--------------+------+



## Feature: isMobile
There's overlapp between this variable & deviceCategory
<br />No NAs

In [25]:
df05.cube("isMobile").count().sort(desc("count")).show()
#717217 is the total number of rows in the United States 

+--------+------+
|isMobile| count|
+--------+------+
|    null|717217|
|   false|475205|
|    true|242012|
+--------+------+



## Feature: operatingSystem
### df05 before this step 

In [26]:
df05.cube("operatingSystem").count().sort(desc("count")).show()

+----------------+------+
| operatingSystem| count|
+----------------+------+
|            null|717217|
|       Macintosh|230058|
|         Windows|160625|
|             iOS|123967|
|         Android|116485|
|           Linux| 46001|
|       Chrome OS| 39310|
|       (not set)|   411|
|   Windows Phone|   163|
|            Xbox|    70|
|      BlackBerry|    38|
|           Tizen|    26|
|    Nintendo Wii|    23|
|   Nintendo WiiU|    14|
|    Nintendo 3DS|     6|
|         FreeBSD|     6|
|      Firefox OS|     5|
|Playstation Vita|     3|
|           SunOS|     3|
|           Nokia|     2|
+----------------+------+
only showing top 20 rows



In [27]:
#to make nullable = True 
df06 = df05.withColumn('Noperatingsystem', fn.when(fn.col('operatingSystem')=='Macintosh', 'Macintosh').\
                                              when(fn.col('operatingSystem')=='Windows', 'Windows').\
                                              when(fn.col('operatingSystem')=='iOS', 'iOS').\
                                              when(fn.col('operatingSystem')=='Android', 'Android').\
                                              otherwise('others')).\
                                              drop('operatingSystem')

In [28]:
df06.cube("Noperatingsystem").count().sort(desc("count")).show()

+----------------+------+
|Noperatingsystem| count|
+----------------+------+
|            null|717217|
|       Macintosh|230058|
|         Windows|160625|
|             iOS|123967|
|         Android|116485|
|          others| 86082|
+----------------+------+



## Feature: country
The dataset is already filtered, drop the column
## Feature: isMobile
There's an overlap between isMobile & device categories, drop the column

In [29]:
df06 = df06.drop('country').drop('isMobile')

In [30]:
#df06.printSchema()

## Feature: totalsbounces
### Before this step, it's df06
Here, each row is by sessions, 
<br />For null values, we fill them with 0. 
<br />Because if there's a bounce, they leave the wesbite. Otherwise, they stays. 

In [31]:
#replace Null with 0 
df07 = df06.withColumn("Ntotalsbounces",fn.when(fn.isnull('totalsbounces') ,0).otherwise(1))
df07 = df07.drop('totalsbounces')

In [32]:
df07.cube("Ntotalsbounces").count().sort(desc("count")).show()
# total number of rows 717217

+--------------+------+
|Ntotalsbounces| count|
+--------------+------+
|          null|717217|
|             0|439702|
|             1|277515|
+--------------+------+



In [33]:
#df07.printSchema()

## Feature: totalshits
 A Hit actually refers to the number of files downloaded on your site, this could include photos, graphics, etc. Picture the average web page, it has photos (each photo is a file and hence a hit) and lots of buttons (each button is a file and hence a hit). On average, each page will include 15 hits.
<br />https://www.tendenci.com/help-files/meaning-of-hits-visits-page-views-and-traffic-sources-web-analytics-definitions/

In [34]:
#feature description 
df07.select('totalshits').describe().show()

+-------+-----------------+
|summary|       totalshits|
+-------+-----------------+
|  count|           717217|
|   mean|6.117884824258209|
| stddev|11.13035020843888|
|    min|                1|
|    max|              500|
+-------+-----------------+



In [35]:
df07.cube("totalshits").count().sort(desc("count")).show()
# total number of rows 717217.
# No nulls 

+----------+------+
|totalshits| count|
+----------+------+
|      null|717217|
|         1|275244|
|         2| 89162|
|         3| 69314|
|         4| 45103|
|         5| 35056|
|         6| 28004|
|         7| 22756|
|         8| 17977|
|         9| 14565|
|        10| 12091|
|        11| 10541|
|        12|  8996|
|        13|  7924|
|        14|  6982|
|        15|  6139|
|        16|  5492|
|        17|  4809|
|        18|  4421|
|        19|  3900|
+----------+------+
only showing top 20 rows



## Feature: totalsnewVisits
### Before this step, it's df07
If the visit is a new visit according to the fullVisitorID, it's 1
<br />If the visit is a not a new one, it's null. 
<br />It's logical to assign value 0 to non-new visits

In [37]:
#replace Nulls with 0
df08 = df07.withColumn("NtotalsnewVisits",fn.when(fn.isnull('totalsnewVisits') ,0).otherwise(1))
df08 = df08.drop('totalsnewVisits')

In [38]:
#check NAs
print(df08.filter(df08["NtotalsnewVisits"].isNull()).count())

0


## Feature: totalspageviews
### Before this step, it's df08
<br />103 Nulls, replace them with 0


In [39]:
#feature description 
print(df08.filter(df08["totalspageviews"].isNull()).count())
print(df08.cube("totalspageviews").count().sort(desc("count")).show())# total number of rows 717217.
print(df08.select('totalspageviews').describe().show())

103
+---------------+------+
|totalspageviews| count|
+---------------+------+
|           null|717217|
|              1|279242|
|              2| 94384|
|              3| 73996|
|              4| 49348|
|              5| 38224|
|              6| 28999|
|              7| 23174|
|              8| 18025|
|              9| 14821|
|             10| 11805|
|             11| 10245|
|             12|  8382|
|             13|  7184|
|             14|  6404|
|             15|  5490|
|             16|  4722|
|             17|  4149|
|             18|  3693|
|             19|  3113|
+---------------+------+
only showing top 20 rows

None
+-------+-----------------+
|summary|  totalspageviews|
+-------+-----------------+
|  count|           717114|
|   mean| 5.01755090543484|
| stddev|7.963383177157497|
|    min|                1|
|    max|              500|
+-------+-----------------+

None


In [40]:
df09 = df08.na.fill({'totalspageviews': 0})
#df09.printSchema()

In [41]:
#check results
print(df09.filter(df09["totalspageviews"].isNull()).count())

0


In [42]:
#description after the replacement
print(df09.select('totalspageviews').describe().show())

+-------+-----------------+
|summary|  totalspageviews|
+-------+-----------------+
|  count|           717217|
|   mean|5.016830331684832|
| stddev|  7.9630383320491|
|    min|                0|
|    max|              500|
+-------+-----------------+

None


## Feature: totalssessionQualityDim
### df09 before this step 
<br />333031 Nulls in totalssessionQualityDim
<br />Replace all Nulls with average 
<br />https://www.optimizesmart.com/understanding-session-quality-data-and-report-in-google-analytics/
<br />This is a metric that was calcualted by google analytics for web usage, it could serve as a aggregated stat for all the web usage matrix

In [43]:
#feature description 
print(df09.filter(df09["totalssessionQualityDim"].isNull()).count())
print(df09.cube("totalssessionQualityDim").count().sort(desc("count")).show())# total number of rows 717217.
print(df09.select('totalssessionQualityDim').describe().show())

333031
+-----------------------+------+
|totalssessionQualityDim| count|
+-----------------------+------+
|                   null|717217|
|                   null|333031|
|                      1|263525|
|                      2| 41341|
|                      3| 12665|
|                      4|  6638|
|                      5|  4549|
|                      6|  3322|
|                      7|  2489|
|                      8|  2126|
|                      9|  1707|
|                     10|  1520|
|                     11|  1348|
|                     12|  1173|
|                     13|  1082|
|                     14|   962|
|                     15|   885|
|                     16|   866|
|                     22|   744|
|                     19|   736|
+-----------------------+------+
only showing top 20 rows

None
+-------+-----------------------+
|summary|totalssessionQualityDim|
+-------+-----------------------+
|  count|                 384186|
|   mean|      6.535063745165102|


In [44]:
# Replace Nulls with 6.535063745165102, the average
df10 = df09.na.fill({'totalssessionQualityDim': 6.535063745165102})
#df10.printSchema()

In [45]:
#check results
print(df10.filter(df10["totalssessionQualityDim"].isNull()).count())
#description after the replacement
print(df10.select('totalssessionQualityDim').describe().show())

0
+-------+-----------------------+
|summary|totalssessionQualityDim|
+-------+-----------------------+
|  count|                 717217|
|   mean|      6.286613395945718|
| stddev|     11.815170812904595|
|    min|                      1|
|    max|                    100|
+-------+-----------------------+

None


## Feature: totalstimeOnSite
### Units in Second; Before this step, it's df10
<br />totalstimeOnSite, there are 278817 nulls. Here it's more reasonal to assume 0 for Nulls, as there are many people boucing off the sites & there are many technical clithces
<br />19017 is the max value, if this is in Seconds, it's 5.28 hours. Otherwise, it's 5 days. It's more rational if the unit is in seconds

In [46]:
#feature description 
print(df10.filter(df10["totalstimeOnSite"].isNull()).count())
print(df10.cube("totalstimeOnSite").count().sort(desc("count")).show())# total number of rows 717217.
print(df10.select('totalstimeOnSite').describe().show())

278817
+----------------+------+
|totalstimeOnSite| count|
+----------------+------+
|            null|717217|
|            null|278817|
|               4|  5021|
|               5|  4697|
|               6|  4672|
|              10|  4419|
|               9|  4369|
|              12|  4363|
|              11|  4339|
|               7|  4252|
|               8|  4246|
|              13|  4214|
|              14|  4129|
|              15|  3951|
|               3|  3937|
|              17|  3888|
|              16|  3883|
|              19|  3703|
|              18|  3679|
|              20|  3532|
+----------------+------+
only showing top 20 rows

None
+-------+------------------+
|summary|  totalstimeOnSite|
+-------+------------------+
|  count|            438400|
|   mean|270.01987910583944|
| stddev|490.17557868806523|
|    min|                 1|
|    max|             19017|
+-------+------------------+

None


In [47]:
# Replace Nulls with 6.535063745165102, the average
df11 = df10.na.fill({'totalstimeOnSite': 0})
#df11.printSchema()

In [48]:
#check results
print(df11.filter(df11["totalstimeOnSite"].isNull()).count())
#description after the replacement
print(df11.select('totalstimeOnSite').describe().show())

0
+-------+------------------+
|summary|  totalstimeOnSite|
+-------+------------------+
|  count|            717217|
|   mean|165.05006852877162|
| stddev|405.20594721375556|
|    min|                 0|
|    max|             19017|
+-------+------------------+

None


## Feature: totalTransactionRevenue
### Before this step, it's df11
<br />699559 Nulls, Replace Nulls with 0 

In [49]:
#feature description 
print(df11.filter(df11["totalTransactionRevenue"].isNull()).count())
print(df11.cube("totalTransactionRevenue").count().sort(desc("count")).show())# total number of rows 717217.
print(df11.select('totalTransactionRevenue').describe().show())

699559
+-----------------------+------+
|totalTransactionRevenue| count|
+-----------------------+------+
|                   null|717217|
|                   null|699559|
|               24990000|   138|
|               23990000|   137|
|               22990000|   128|
|               21990000|   113|
|               25990000|   112|
|               20990000|    93|
|               19990000|    85|
|               26990000|    84|
|               18990000|    81|
|               27990000|    81|
|               28990000|    77|
|               17990000|    73|
|               16990000|    58|
|               45980000|    54|
|               29990000|    53|
|               44980000|    49|
|               40980000|    49|
|               20590000|    47|
+-----------------------+------+
only showing top 20 rows

None
+-------+-----------------------+
|summary|totalTransactionRevenue|
+-------+-----------------------+
|  count|                  17658|
|   mean|   1.4130140446256655E8|


In [50]:
# Replace Nulls with 6.535063745165102, the average
df12 = df11.na.fill({'totalTransactionRevenue': 0})
#df12.printSchema()

In [51]:
#check results
print(df12.filter(df12["totalTransactionRevenue"].isNull()).count())
#description after the replacement
print(df12.select('totalTransactionRevenue').describe().show())

0
+-------+-----------------------+
|summary|totalTransactionRevenue|
+-------+-----------------------+
|  count|                 717217|
|   mean|      3478863.719069682|
| stddev|    1.046079873095143E8|
|    min|                      0|
|    max|            47082060000|
+-------+-----------------------+

None


## Feature: transactionRevenue
### Before this step, it's df12
<br />699559 Nulls, Replace Nulls with 0 

In [52]:
#feature description 
print(df12.filter(df12["transactionRevenue"].isNull()).count())
print(df12.cube("transactionRevenue").count().sort(desc("count")).show())# total number of rows 717217.
print(df12.select('transactionRevenue').describe().show())

699559
+------------------+------+
|transactionRevenue| count|
+------------------+------+
|              null|717217|
|              null|699559|
|          16990000|   281|
|          19990000|   240|
|          39980000|   214|
|          18990000|   213|
|          21990000|   204|
|          33590000|   200|
|          17590000|   185|
|          44790000|   182|
|          13590000|   177|
|          10990000|   155|
|          15990000|   151|
|          41590000|   148|
|          55990000|   147|
|          79990000|   147|
|          19190000|   144|
|          35180000|   114|
|          15190000|   113|
|          59990000|   112|
+------------------+------+
only showing top 20 rows

None
+-------+--------------------+
|summary|  transactionRevenue|
+-------+--------------------+
|  count|               17658|
|   mean|1.2446965624646053E8|
| stddev|4.2080082907579446E8|
|    min|               10000|
|    max|         23129500000|
+-------+--------------------+

None


In [53]:
# Replace Nulls with 6.535063745165102, the average
df13 = df12.na.fill({'transactionRevenue': 0})
#df13.printSchema()

In [54]:
#check results
print(df13.filter(df13["transactionRevenue"].isNull()).count())
#description after the replacement
print(df13.select('transactionRevenue').describe().show())

0
+-------+-------------------+
|summary| transactionRevenue|
+-------+-------------------+
|  count|             717217|
|   mean|   3064463.32142155|
| stddev|6.878499867835435E7|
|    min|                  0|
|    max|        23129500000|
+-------+-------------------+

None


## Feature: transactions
### Before this step, it's df13
<br/>699517 Nulls, Replace Nulls with 0 

In [55]:
#feature description 
print(df13.filter(df12["transactions"].isNull()).count())
print(df13.cube("transactions").count().sort(desc("count")).show())# total number of rows 717217.
print(df13.select('transactions').describe().show())

699517
+------------+------+
|transactions| count|
+------------+------+
|        null|717217|
|        null|699517|
|           1| 17238|
|           2|   390|
|           3|    39|
|           4|    11|
|           5|     9|
|           6|     4|
|           7|     3|
|           8|     2|
|          12|     2|
|          25|     1|
|          15|     1|
+------------+------+

None
+-------+------------------+
|summary|      transactions|
+-------+------------------+
|  count|             17700|
|   mean|1.0366666666666666|
| stddev|0.3432415390497413|
|    min|                 1|
|    max|                25|
+-------+------------------+

None


In [56]:
# Replace Nulls with 6.535063745165102, the average
df14 = df13.na.fill({'transactions': 0})
#df14.printSchema()

In [57]:
#check results
print(df14.filter(df14["transactions"].isNull()).count())
#description after the replacement
print(df14.select('transactions').describe().show())

0
+-------+-------------------+
|summary|       transactions|
+-------+-------------------+
|  count|             717217|
|   mean| 0.0255836099813585|
| stddev|0.16963062273722684|
|    min|                  0|
|    max|                 25|
+-------+-------------------+

None


## Feature: totalsvisits
### All entries are 1, the feature is useless
Feature Dropped
<br/>0 Nulls

In [58]:
#feature description 
print(df14.filter(df14["totalsvisits"].isNull()).count())
print(df14.cube("totalsvisits").count().sort(desc("count")).show())# total number of rows 717217.
print(df14.select('totalsvisits').describe().show())

0
+------------+------+
|totalsvisits| count|
+------------+------+
|           1|717217|
|        null|717217|
+------------+------+

None
+-------+------------+
|summary|totalsvisits|
+-------+------------+
|  count|      717217|
|   mean|         1.0|
| stddev|         0.0|
|    min|           1|
|    max|           1|
+-------+------------+

None


In [59]:
df14 = df14.drop('totalsvisits')
#df14.printSchema()

## Feature: adwordsClickInfoisVideoAd
### 655472 Nulls. Before this step, it's df14
Other entries are false, I.e. these are not clicked through a ad. 
<br/> We'll asign 1 to Null, 0 to False
<br/> 

In [60]:
#feature description 
print(df14.filter(df14["adwordsClickInfoisVideoAd"].isNull()).count())
print(df14.cube("adwordsClickInfoisVideoAd").count().sort(desc("count")).show())# total number of rows 717217.
# print(df14.select('adwordsClickInfoisVideoAd').describe().show())

655472
+-------------------------+------+
|adwordsClickInfoisVideoAd| count|
+-------------------------+------+
|                     null|717217|
|                     null|655472|
|                    false| 61745|
+-------------------------+------+

None


In [61]:
#df14 = df14.na.fill({'adwordsClickInfoisVideoAd': 0})
df15 = df14.withColumn("NadwordsClickInfoisVideoAd",fn.when(fn.isnull('adwordsClickInfoisVideoAd'), 0).otherwise(1))
df15 = df15.drop('adwordsClickInfoisVideoAd')

In [62]:
#check results
print(df15.filter(df15["NadwordsClickInfoisVideoAd"].isNull()).count())
#description after the replacement
print(df15.select('NadwordsClickInfoisVideoAd').describe().show())
print(df15.cube("NadwordsClickInfoisVideoAd").count().sort(desc("count")).show())# total number of rows 717217.

0
+-------+--------------------------+
|summary|NadwordsClickInfoisVideoAd|
+-------+--------------------------+
|  count|                    717217|
|   mean|       0.08608970506834054|
| stddev|        0.2804966620998005|
|    min|                         0|
|    max|                         1|
+-------+--------------------------+

None
+--------------------------+------+
|NadwordsClickInfoisVideoAd| count|
+--------------------------+------+
|                      null|717217|
|                         0|655472|
|                         1| 61745|
+--------------------------+------+

None


## Feature: isTrueDirect
### 419509 Nulls. Before this step, it's df15
Other entries are false, I.e. these are not clicked through a ad. 
<br/> We'll asign 0 to Null, 1 to False
<br/> This feature is higly correlated with the channel group, because direction was already included
https://support.google.com/analytics/answer/3437719?hl=en Same link is posted on both here & google doc

In [63]:
#feature description 
print(df15.filter(df15["isTrueDirect"].isNull()).count())
print(df15.cube("isTrueDirect").count().sort(desc("count")).show())# total number of rows 717217.
# print(df15.select('isTrueDirect').describe().show())

419509
+------------+------+
|isTrueDirect| count|
+------------+------+
|        null|717217|
|        null|419509|
|        true|297708|
+------------+------+

None


In [64]:
#df16 = df15.na.fill({'isTrueDirect': 0})
df16 = df15.withColumn("NisTrueDirect",fn.when(fn.isnull('isTrueDirect'), 0).otherwise(1))
df16 = df16.drop('isTrueDirect')

In [65]:
#check results
print(df16.filter(df16["NisTrueDirect"].isNull()).count())
#description after the replacement
#print(df16.select('NisTrueDirect').describe().show())
print(df16.cube("NisTrueDirect").count().sort(desc("count")).show())# total number of rows 717217.

0
+-------------+------+
|NisTrueDirect| count|
+-------------+------+
|         null|717217|
|            0|419509|
|            1|297708|
+-------------+------+

None


## Feature: trafficSourcemedium
### 0  Nulls. Before this step, it's df16
There are (none) & (not set)
<br/> 

In [66]:
#feature description 
print(df16.filter(df16["trafficSourcemedium"].isNull()).count())
print(df16.cube("trafficSourcemedium").count().sort(desc("count")).show())# total number of rows 717217.
# print(df15.select('isTrueDirect').describe().show())

0
+-------------------+------+
|trafficSourcemedium| count|
+-------------------+------+
|               null|717217|
|             (none)|334158|
|            organic|236304|
|           referral| 72393|
|                cpc| 58902|
|                cpm|  8372|
|          affiliate|  6985|
|          (not set)|   103|
+-------------------+------+

None


In [67]:
df17 = df16.withColumn('NtrafficSourcemedium', fn.when(fn.col('trafficSourcemedium')=='organic', 'organic').\
                                               when(fn.col('trafficSourcemedium')=='referral', 'referral').\
                                               when(fn.col('trafficSourcemedium')=='cpc', 'cpc').\
                                               when(fn.col('trafficSourcemedium')=='cpm', 'cpm').\
                                               when(fn.col('trafficSourcemedium')=='affiliate', 'affiliate').\
                                               otherwise('others')).\
                                               drop('trafficSourcemedium')

In [68]:
#check results
print(df17.filter(df17["NtrafficSourcemedium"].isNull()).count())
#description after the replacement
#print(df16.select('NisTrueDirect').describe().show())
print(df17.cube("NtrafficSourcemedium").count().sort(desc("count")).show())# total number of rows 717217.

0
+--------------------+------+
|NtrafficSourcemedium| count|
+--------------------+------+
|                null|717217|
|              others|334261|
|             organic|236304|
|            referral| 72393|
|                 cpc| 58902|
|                 cpm|  8372|
|           affiliate|  6985|
+--------------------+------+

None


## Feature: trafficSourcesource
### 0  Nulls. Before this step, it's df17
<br/> 
<br/> 

In [69]:
#feature description 
print(df17.filter(df17["trafficSourcesource"].isNull()).count())
print(df17.cube("trafficSourcesource").count().sort(desc("count")).show())# total number of rows 717217.
# print(df15.select('isTrueDirect').describe().show())

0
+--------------------+------+
| trafficSourcesource| count|
+--------------------+------+
|                null|717217|
|            (direct)|334176|
|              google|293586|
|         youtube.com| 35234|
|    sites.google.com|  8152|
|analytics.google.com|  7130|
|            Partners|  6988|
|                 dfa|  6114|
|      m.facebook.com|  2751|
|          google.com|  2726|
|     mail.google.com|  1910|
|   groups.google.com|  1822|
|          reddit.com|  1726|
|                bing|  1711|
|siliconvalley.abo...|  1389|
|               yahoo|  1214|
|googleads.g.doubl...|  1040|
|        facebook.com|  1010|
|                t.co|   886|
|               baidu|   783|
+--------------------+------+
only showing top 20 rows

None


In [70]:
df18 = df17.withColumn('NtrafficSourcesource', 
                       fn.when(fn.col('trafficSourcesource')=='google', 'google').\
                          when(fn.col('trafficSourcesource')=='sites.google.com', 'google').\
                          when(fn.col('trafficSourcesource')=='analytics.google.com', 'google').\
                          when(fn.col('trafficSourcesource')=='google.com', 'google').\
                          when(fn.col('trafficSourcesource')=='mail.google.com', 'google').\
                          when(fn.col('trafficSourcesource')=='groups.google.com', 'google').\
                          when(fn.col('trafficSourcesource')=='mail.google.com', 'google').\
                          when(fn.col('trafficSourcesource')=='groups.google.com', 'google').\
                          when(fn.col('trafficSourcesource')=='youtube.com', 'youtube').\
                               otherwise('others')).\
                               drop('trafficSourcemedium')

In [71]:
#check results
print(df18.filter(df18["NtrafficSourcesource"].isNull()).count())
#description after the replacement
#print(df16.select('NisTrueDirect').describe().show())
print(df18.cube("NtrafficSourcesource").count().sort(desc("count")).show())# total number of rows 717217.

0
+--------------------+------+
|NtrafficSourcesource| count|
+--------------------+------+
|                null|717217|
|              others|366657|
|              google|315326|
|             youtube| 35234|
+--------------------+------+

None


# df18 is the final dataset after all the adjustment 
<br/> 

In [72]:
df18.printSchema()

root
 |-- channelGrouping: string (nullable = true)
 |-- fullVisitorId: decimal(20,0) (nullable = true)
 |-- visitId: integer (nullable = true)
 |-- visitNumber: integer (nullable = true)
 |-- deviceCategory: string (nullable = true)
 |-- totalshits: integer (nullable = true)
 |-- totalspageviews: integer (nullable = false)
 |-- totalssessionQualityDim: integer (nullable = true)
 |-- totalstimeOnSite: integer (nullable = false)
 |-- totalTransactionRevenue: long (nullable = false)
 |-- transactionRevenue: long (nullable = false)
 |-- transactions: integer (nullable = false)
 |-- trafficSourcesource: string (nullable = true)
 |-- Nbrowsers: string (nullable = false)
 |-- Noperatingsystem: string (nullable = false)
 |-- Ntotalsbounces: integer (nullable = false)
 |-- NtotalsnewVisits: integer (nullable = false)
 |-- NadwordsClickInfoisVideoAd: integer (nullable = false)
 |-- NisTrueDirect: integer (nullable = false)
 |-- NtrafficSourcemedium: string (nullable = false)
 |-- NtrafficSource

In [77]:
df18.toPandas().to_csv('Step02_updated02.csv',header = True)
# Output this dataset